In [1]:
import pandas as pd
import numpy as np

In [2]:
dt= pd.read_csv('train_tweet.csv')

In [3]:
dt['polarity_number']= np.NaN
dt.loc[dt['polarity']=='P', 'polarity_number' ]= 1
dt.loc[dt['polarity']=='N', 'polarity_number' ]= 0
dt.loc[dt['polarity']=='NEU', 'polarity_number' ]= np.NAN
dt.loc[dt['polarity']=='NONE', 'polarity_number' ]= np.NAN

In [4]:
dt=dt.dropna()

In [5]:
dt= dt[['content' , 'polarity_number']]

In [6]:
from gensim.utils import simple_preprocess ,tokenize
from gensim.models  import KeyedVectors

In [7]:
model_w2v = KeyedVectors.load_word2vec_format('SBW-vectors-300-min5.bin.gz', binary=True)

In [8]:
lista_index= []
for i in dt['content']:
    a = simple_preprocess(i)
    lista_index.append([ model_w2v.vocab[w].index for w in a if w in model_w2v.vocab])


In [10]:
sentences= lista_index
y = dt['polarity_number'].values

In [24]:
len(lista_index), len(y)

(736, 736)

In [11]:
from sklearn.model_selection import train_test_split


In [12]:
sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, test_size=0.25, random_state=1000)

In [13]:
from keras.preprocessing.sequence import pad_sequences


Using TensorFlow backend.


In [14]:
maxlen= 30
X_train = pad_sequences(sentences_train, padding='post', maxlen=maxlen)


In [63]:
print(X_train[0])

[398481      2  35870      2   1806    561   1473     38    714     19
   2867     53    254  37584    189  48010      0      0      0      0
      0      0      0      0      0      0      0      0      0      0]


In [27]:
X_test = pad_sequences(sentences_test, padding='post', maxlen=maxlen)

In [28]:
print(len(X_train),len(y_train),len(X_test),len(y_test))

552 552 184 184


In [16]:
embedding_dim = model_w2v.get_keras_embedding(train_embeddings=False)

In [17]:
vocab_size = len(model_w2v.wv.vocab)


C:\Users\Usuario\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [19]:
from keras.models import Sequential
from keras import layers

In [29]:
embedding_dim = 300

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 30, 300)           300195900 
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 26, 128)           192128    
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
Total params: 300,389,329
Trainable params: 300,389,329
Non-trainable params: 0
_________________________________________________________________


In [33]:
history = model.fit(X_train, y_train,
                    epochs=50,
                    verbose=10,
                    validation_data=(X_test, y_test),
                    batch_size=1
                    0)
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


Train on 552 samples, validate on 184 samples
Epoch 1/50
Epoch 2/50
Epoch 3/50
Epoch 4/50
Epoch 5/50
Epoch 6/50
Epoch 7/50
Epoch 8/50
Epoch 9/50
Epoch 10/50
Epoch 11/50
Epoch 12/50
Epoch 13/50
Epoch 14/50
Epoch 15/50
Epoch 16/50
Epoch 17/50
Epoch 18/50
Epoch 19/50
Epoch 20/50
Epoch 21/50
Epoch 22/50
Epoch 23/50
Epoch 24/50
Epoch 25/50
Epoch 26/50
Epoch 27/50
Epoch 28/50
Epoch 29/50
Epoch 30/50
Epoch 31/50
Epoch 32/50
Epoch 33/50
Epoch 34/50
Epoch 35/50
Epoch 36/50
Epoch 37/50
Epoch 38/50
Epoch 39/50
Epoch 40/50
Epoch 41/50
Epoch 42/50
Epoch 43/50
Epoch 44/50
Epoch 45/50
Epoch 46/50
Epoch 47/50
Epoch 48/50
Epoch 49/50
Epoch 50/50
Training Accuracy: 1.0000
Testing Accuracy:  0.6630


KeyError: 'acc'

In [160]:
a = ['el machine learning es una mierda']

In [161]:
a

['el machine learning es una mierda']

In [162]:
lista_prueba_mal= []
for i in a:
    print(i)
    a = simple_preprocess(i)
    lista_prueba_mal.append([ model_w2v.vocab[w].index for w in a if w in model_w2v.vocab])
print(lista_prueba_mal)

el machine learning es una mierda
[[4, 72103, 47730, 20, 16, 32229]]


In [163]:
a_test_matrix= pad_sequences(lista_prueba_mal, padding='post', maxlen=maxlen)

In [165]:
print(a_test_matrix)

[[    4 72103 47730    20    16 32229     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0]]


In [166]:
model.predict_classes(a_test_matrix),model.predict_proba(a_test_matrix)

(array([[0]]), array([[0.00022429]], dtype=float32))

In [167]:
b= ['el machine learning mola']

In [168]:
b

['el machine learning mola']

In [169]:
lista_prueba_bien= []
for i in b:
    print(i)
    a = simple_preprocess(i)
    lista_prueba_bien.append([ model_w2v.vocab[w].index for w in a if w in model_w2v.vocab])
print(lista_prueba_bien)

el machine learning mola
[[4, 72103, 47730, 116376]]


In [170]:
b_test_matrix= pad_sequences(lista_prueba_bien, padding='post', maxlen=maxlen)

In [171]:
b_test_matrix

array([[     4,  72103,  47730, 116376,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0]])

In [172]:
model.predict_classes(b_test_matrix),model.predict_proba(b_test_matrix)

(array([[1]]), array([[0.979606]], dtype=float32))

In [205]:
def prediction(texto):
    maxlen=30
    prediction_matrix=[]
    a = simple_preprocess(texto)
    prediction_matrix.append([ model_w2v.vocab[w].index for w in a if w in model_w2v.vocab])
    print(prediction_matrix)
    prediction_matrix= pad_sequences(prediction_matrix, padding='post', maxlen=maxlen)
    class_= model.predict_classes(prediction_matrix)
    probability= model.predict_proba(prediction_matrix)
    print(probability)
    return class_

In [210]:
prediction('el machine learning es una mierda')

[[4, 72103, 47730, 20, 16, 32229]]
[[0.00022429]]


array([[0]])

In [211]:
prediction('el machine learning mola')

[[4, 72103, 47730, 116376]]
[[0.979606]]


array([[1]])

In [222]:
prediction('me da mucho gusto')

[[189, 527, 283, 4049]]
[[0.99629813]]


array([[1]])

In [190]:
predicciones= []
for i in dt['content']:
    predicciones.append(prediction(i))

[[6, 109073, 1181, 29381, 189, 18160, 38136, 30, 2969, 10079, 4, 1309, 398481]]
[[398481, 2, 35870, 2, 1806, 561, 1473, 38, 714, 19, 2867, 53, 254, 37584, 189, 48010]]
[[1863, 170757, 38, 23475, 12, 26, 2259, 6, 20631, 8]]
[[5699, 937, 1168, 2, 141530, 17, 4043, 189, 42888, 621, 968354, 187445]]
[[39437, 283, 24, 12, 1761, 323, 6, 26, 54533, 189, 68894, 8900, 1181]]
[[11, 30, 3840, 28029, 10156, 19, 44997, 10, 4872, 18, 403, 3944, 12, 2, 42267, 6, 187, 763, 2653, 3038]]
[[31, 18387, 32639, 156602, 6, 39435, 10692, 16, 4437]]
[[6325, 210, 6, 67565, 0, 5952, 153, 5438, 48, 105, 6, 21803, 15, 151, 392]]
[[1072, 235144, 13, 31514, 38, 19, 1390, 6, 1001, 3473, 5431, 219]]
[[6697, 24549, 13, 23050, 18387, 76837, 13978, 140]]
[[2, 1347, 0, 84874, 18, 231, 189, 9457, 35265, 714, 6]]
[[299619, 20, 0, 151, 6360, 45, 25273, 6, 10255, 1597, 63, 4, 3433, 0, 408, 9, 1390, 6756, 14, 26, 6, 78]]
[[2405, 18, 141, 6203, 6830, 2, 609, 14, 16, 8515, 726, 112530, 18641, 189, 3657, 23611, 12, 2, 609]]
[[259

[[38, 197, 709, 76916, 24978, 862, 4489, 12311, 20221, 10, 1434, 71, 197, 429]]
[[3, 476, 3489, 0, 27924, 172, 4, 4074, 189, 441]]
[[8, 228947, 57, 111, 24466, 28, 65, 120, 2, 318, 19, 3883, 25836]]
[[392, 63, 6, 45, 69, 13067, 189, 28517, 6, 8, 471, 111, 95623, 19]]
[[72207, 4228, 1863, 287, 8, 6, 189, 19632, 5752]]
[[3, 805, 20, 35085, 65, 392, 22, 2, 7361, 153, 0, 9, 38, 1390, 6, 3, 2, 132, 7707, 476, 142]]
[[78, 13, 549, 3, 2, 746, 387913, 714, 51, 2, 6124, 42, 937, 467, 343, 189, 22722, 1181, 2512, 14408]]
[[189, 64613, 3, 431, 19, 189, 355, 189, 836261, 15, 35801, 6, 189, 158126, 1168]]
[[57, 20, 6, 19, 2867, 5483, 59, 1072, 32873, 2724, 6, 75146, 19, 1072, 2867, 5483, 19, 26, 17216, 4145, 237790]]
[[3, 10, 3062, 20, 18, 649481, 24, 544, 24, 3961, 38, 4688, 93, 26, 27878, 20, 16, 395, 155, 11, 24730]]
[[2057, 209, 4900, 210648]]
[[151, 151, 57, 22618, 10292, 13, 8335, 57, 58, 12194, 4, 93, 16538, 18, 141, 39, 37, 5220]]
[[541237, 16, 16516, 11342, 3, 473, 12, 69, 9, 418489, 6, 30

[[197, 709, 682, 142, 220, 7825, 36, 990, 5358, 4, 1428]]
[[189, 2010, 69, 20896, 6, 2, 561, 19, 21999, 6, 449, 2867, 61620]]
[[664, 6, 1510, 38, 2, 1892, 235]]
[[926218, 41, 19, 937, 18498, 1554, 15, 6, 1646, 47, 0, 28712]]
[[254, 189, 753, 44156, 38, 89999, 2638, 10, 953, 17977]]
[[20, 6, 459502, 47, 1028, 16, 1541, 45241]]
[[15, 45, 13, 562056, 63, 1563, 43239, 57, 20, 69, 2405, 4209, 18, 116191, 14, 4, 454718, 8, 835, 562056]]
[[664, 5681, 6, 29558, 16, 673, 421]]
[[28110, 19, 709, 570, 28, 8, 120, 19, 41, 3124, 1653, 50, 2, 3415, 91329, 26, 93, 50, 351]]
[[189, 7915, 552, 859082, 0, 58, 5486, 4, 393, 0, 19, 189, 3657, 570]]
[[189, 5931, 11264, 53103, 258438, 19, 3, 1551, 19, 324, 20, 16, 32229]]
[[5699, 48, 4044, 202931, 937, 771, 6, 1910, 4, 131172, 0, 2, 1827, 12, 4, 0, 2, 1571, 240, 10019]]
[[1396, 19, 6574, 196, 14, 24969, 5956, 19, 8, 181, 3, 545, 280, 61043]]
[[359, 5948, 16, 673, 8460, 4209, 289, 151]]
[[151, 714, 19, 11, 6, 63, 4, 148, 3, 52, 0, 2, 1076, 1888, 254, 20, 2, 

[[56421, 392, 4, 49, 214, 254, 3634, 11, 319, 4, 11165, 14, 2, 8084]]
[[202, 14018, 1137, 18, 9, 1387, 12, 1910, 369, 4, 348, 20, 210]]
[[57, 48, 10, 8645, 38, 68169, 20, 2067, 8597, 125, 13, 10100, 3, 296]]
[[22, 289, 254, 22754, 30539, 2353, 6, 62, 7054, 3023, 30536, 5714, 173452]]
[[3842, 57197, 32472, 506, 664]]
[[189, 15028, 22, 40, 2513, 6, 2829, 3460, 1137, 53, 172, 2872]]
[[15, 2067, 6279, 155, 272752, 2729, 19, 8913, 58891, 85, 18, 533, 9, 160, 15, 8, 62, 2067, 2916]]
[[3, 1551, 369, 38, 714, 16309, 7213, 414, 449, 37108, 34502]]
[[1542, 339297, 3323, 9, 3578, 12313, 3578]]
[[714, 206, 196297, 57657, 206, 264760, 59025, 120, 25029, 20, 26, 6, 1674, 59, 83105, 4, 22543, 22, 13, 2752]]
[[0, 1032, 32629, 86, 0, 16, 438, 6623]]
[[289, 10, 835, 70, 39, 10, 6, 24, 5020]]
[[20, 1563, 48210, 28, 65, 6, 26, 2061, 561, 6, 2330, 19, 189, 1911, 0, 570]]
[[26, 628, 7078, 0, 27006, 34165, 20, 6, 3669, 69, 296]]
[[20, 6, 10, 2281, 249523, 19, 111, 151, 7405, 69, 28473, 296, 7470]]
[[22, 11, 

[[119297, 88364, 4, 1942, 0, 1532, 391167, 2384, 45, 2259, 3, 5049, 2505, 22372, 7905, 380]]
[[359, 254, 754, 12, 697, 19, 937, 6227, 38, 254, 101, 369, 3, 490, 57, 6574, 2, 24728, 995, 0, 3604, 8783, 39588, 4688, 65]]
[[0, 271637, 150298, 322, 599180]]
[[120, 12, 19, 123205, 14, 13, 5791, 558, 189, 937, 10611, 2, 60329, 4, 139, 6, 189, 1478, 632, 20, 0, 189852]]
[[20, 6, 11, 10958, 91, 1666, 141, 1181, 6, 88, 26, 7301, 0, 613, 59, 2130, 2, 459, 2112, 254, 343, 20, 26, 670]]
[[2067, 2, 19867, 5660, 0, 6, 3, 243, 1058, 323, 73, 448, 10, 3419, 9, 234218]]
[[2, 313275, 35164, 4, 2313, 0, 88949, 3, 6, 23776, 10, 19343, 53, 7868, 28, 26, 6, 2965]]
[[57, 257809, 5398, 42, 5001, 143, 108, 17, 276, 48, 4024, 4, 227, 29314, 142, 283]]
[[189506, 206357, 12878, 48, 10, 937, 1168, 38, 9006, 2, 374, 6, 189, 1835, 402, 24086, 30089, 71, 6, 12, 365, 3460, 664]]
[[494, 0, 309, 9333, 254, 139, 295, 3, 92, 0, 3181, 48, 2067, 6730, 0, 6, 26, 278755]]
[[145870, 45665, 664, 12, 2, 28110, 26, 2405, 541, 997

In [192]:
len(predicciones)

736

In [193]:
len(dt['content'])

736

In [194]:
dt['predicciones']= predicciones

In [195]:
dt

,content,polarity_number,prediction,predicciones
1,@myendlesshazza a. que puto mal escribo\r\n\r\...,0.0,None,[[0]]
2,@estherct209 jajajaja la tuya y la d mucha gen...,0.0,None,[[0]]
3,Quiero mogollón a @AlbaBenito99 pero sobretodo...,1.0,None,[[1]]
4,Vale he visto la tia bebiendose su regla y me ...,0.0,None,[[0]]
5,@Yulian_Poe @guillermoterry1 Ah. mucho más por...,1.0,None,[[1]]
...,...,...,...,...
1002,están todos muy sad hoy no,0.0,None,[[0]]
1003,Para mí mi mejor año fue el 2015,1.0,None,[[1]]
1004,Hoy va a ser un dia muy largo...,0.0,None,[[0]]
1006,Es que son muy bonitas jo,1.0,None,[[1]]


In [223]:
import pickle

In [225]:
filename = 'Keras_model_with_stop_words_0_67acc.sav'
pickle.dump(model, open(filename, 'wb'))